In [1]:
import os
from dataset_operations import generate_tiles, load_inputs, InputDataset,get_transform,read_mean_std,custom_collate_fn,calculate_mean_std
from utils import imread
import pandas as pd
from torch.utils.data import DataLoader

parent_dir = "rois2/"
img_dir = parent_dir + "images/"
label_dir = parent_dir + "labels/"

model_dir = parent_dir + "models/"
false_positives_dir = parent_dir + "tiles/false_positives/"
false_positives_images_dir = false_positives_dir + 'images/'
false_positives_labels_dir = false_positives_dir + 'labels/'

false_negatives_dir = parent_dir + "tiles/false_negatives/"
false_negatives_images_dir = false_negatives_dir + 'images/'
false_negatives_labels_dir = false_negatives_dir + 'labels/'

negatives_dir = parent_dir + "tiles/negatives/"
negatives_images_dir = negatives_dir + 'images/'
negatives_labels_dir = negatives_dir + 'labels/'

tile_dir = parent_dir + "tiles/"
train_tile_dir = tile_dir + "train/"
val_tile_dir = tile_dir + "val/"


In [2]:
os.makedirs(false_positives_dir, exist_ok=True)
os.makedirs(false_positives_images_dir, exist_ok=True)
os.makedirs(false_positives_labels_dir, exist_ok=True)

os.makedirs(false_negatives_dir, exist_ok=True)
os.makedirs(false_negatives_images_dir, exist_ok=True)
os.makedirs(false_negatives_labels_dir, exist_ok=True)

os.makedirs(negatives_dir, exist_ok=True)
os.makedirs(negatives_images_dir, exist_ok=True)
os.makedirs(negatives_labels_dir, exist_ok=True)


In [3]:
# clean & reset negative tile image directory
for file in os.listdir(false_positives_images_dir):
    os.remove(false_positives_images_dir + file)
    
for file in os.listdir(false_negatives_images_dir):
    os.remove(false_negatives_images_dir + file)
    
for file in os.listdir(negatives_images_dir):
    os.remove(negatives_images_dir + file)
    
for file in os.listdir(negatives_labels_dir):
    os.remove(negatives_labels_dir + file)

mine hard negative sample

In [4]:
# take the top 10% high-score ones of the false positives and generate tiles 

false_pos_roi_paths = []

for dirpath, dirnames, filenames in os.walk(false_positives_labels_dir):
    for filename in filenames:
        if filename.lower().endswith('.txt'):
            roi_name = filename.split('.')[0]+'.png'
            roi_path = img_dir + roi_name
            false_pos_roi_paths.append(roi_path) 
print(len(false_pos_roi_paths))

false_neg_roi_paths = []

for dirpath, dirnames, filenames in os.walk(false_negatives_labels_dir):
    for filename in filenames:
        if filename.lower().endswith('.txt'):
            roi_name = filename.split('.')[0]+'.png'
            roi_path = img_dir + roi_name
            false_neg_roi_paths.append(roi_path) 
print(len(false_neg_roi_paths))

        
    

334
334


In [5]:
false_pos_df = generate_tiles(roi_fps=false_pos_roi_paths,roi_label_dir=None, tile_label_dir= false_positives_labels_dir,tile_dir = negatives_dir, negatives_only=True,negative_label_ratio=0.05,negatives_boxes=True,to_csv=False)
    

100%|██████████| 334/334 [1:12:02<00:00, 12.94s/it]


In [6]:
false_neg_df = generate_tiles(roi_fps=false_neg_roi_paths,roi_label_dir=None, tile_label_dir = false_negatives_labels_dir,tile_dir= negatives_dir, negatives_only=False,negative_ratio=0,negative_label_ratio=1,negatives_boxes=False,to_csv=False)
 

100%|██████████| 334/334 [02:58<00:00,  1.87it/s]


In [7]:
negative_inputs = load_inputs(negatives_dir)
print(len(negative_inputs))

710619
710619


In [8]:
train_inputs = load_inputs(train_tile_dir)
val_inputs = load_inputs(val_tile_dir)
full_train_inputs = train_inputs + negative_inputs

28621
6140


In [9]:
# # test for iterating the images in full_train_inputs
# for item in full_train_inputs:
#     img_fp = item['image']
#     print(img_fp)
#     img = imread(img_fp)

In [9]:
mean, std = read_mean_std(parent_dir + 'images/')
print('mean: ', mean)
print('std: ', std)    
    
full_dataset = InputDataset(full_train_inputs, mean,std,'A')
full_dataloader = DataLoader(full_dataset, batch_size=64, shuffle=True, num_workers=0,collate_fn=custom_collate_fn)

print(len(full_dataloader))

mean:  [0.77296131, 0.71338682, 0.68258603]
std:  [0.16751114, 0.19680141, 0.21965831]
mean:  [0.77296131, 0.71338682, 0.68258603]
std:  [0.16751114, 0.19680141, 0.21965831]
11551


In [10]:
import pickle
pickle.dump(full_dataloader, open(model_dir + "full_dataloader.pkl", "wb"))